In [7]:
import os
import rasterio

In [28]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import random
from PIL import Image,ImageFilter

def __blur(img):
    # if img.mode == 'RGB':
    img = img.filter(ImageFilter.GaussianBlur(radius=random.random()))
    return img

def __rotate(img, degree):
    if img.mode =='RGB':
        # set img padding == 128
        img2 = img.convert('RGBA')
        rot = img2.rotate(degree,expand=1)
        fff = Image.new('RGBA', rot.size, (128,) * 4)  # 灰色
        out = Image.composite(rot, fff, rot)
        img = out.convert(img.mode)
        return img
    else:
        # set label padding == 0
        img2 = img.convert('RGBA')
        rot = img2.rotate(degree,expand=1)
        # a white image same size as rotated image
        fff = Image.new('RGBA', rot.size, (255,) * 4)
        # create a composite image using the alpha layer of rot as a mask
        out = Image.composite(rot, fff, rot)
        img = out.convert(img.mode)
        return img




img = Image.open(r'/home/skm/SKM16/Tmp/Stanet4band/SplitTrainValTest/test/A/img_stack_0_188.tif')#.convert('RGB')
# transform = transforms.Grayscale(1)
# osize = [128, 128]
# method=Image.BICUBIC
transform = transforms.Lambda(lambda img: __blur(img))
transform = transforms.ToTensor()
img = transform(img)
# transform = transforms.Lambda(lambda img: __rotate(img, 5))
transform = transforms.Normalize((0.5, 0.5, 0.5),
                                 (0.5, 0.5, 0.5))
img = transform(img)

print(np.array(img).shape)
# img.show()
# img.filter(ImageFilter.GaussianBlur(radius=random.random()))
img

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

In [8]:
fp_img = r"C:\Users\SkyMap\Desktop\J4.tif"
fp_out1 = r"E:\WorkSpaceTest\STANet\samples\A\test_2_0000_0000.tif"
fp_out2 = r"E:\WorkSpaceTest\STANet\samples\B\test_2_0000_0000.tif"
src = rasterio.open(fp_img)
meta = src.meta

mask1 = src.read([1,2,3])
mask2 = src.read([4,5,6])

In [9]:
meta.update({'count':3})
with rasterio.open(fp_out1, 'w', **meta) as dst:
    dst.write(mask1)
with rasterio.open(fp_out2, 'w', **meta) as dst:
    dst.write(mask2)

In [1]:
import torch

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "c:\Users\SkyMap\anaconda3\envs\test_env\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

## Train

In [ ]:
python -m visdom.server

python train.py 
--save_epoch_freq 1 
--angle 15 
--dataroot E:\WorkSpaceSkyMap\Change_detection_Dubai\Data\data_train_model_STANet\SplitTrainVal_verByte\train 
--val_dataroot E:\WorkSpaceSkyMap\Change_detection_Dubai\Data\data_train_model_STANet\SplitTrainVal_verByte\val 
--name DUBAI-CDF0 
--lr 0.001 
--model CDF0 
--batch_size 15 
--load_size 256 
--crop_size 256 
--preprocess rotate_and_crop

# Các config train khac nhau

In [ ]:
python ./train.py --name LEVIR-CDF0   --model CDF0
python ./train.py --name LEVIR-CDFA0  --model CDFA --SA_mode BAM
python ./train.py --name LEVIR-CDFAp0 --model      --SA_mode PAM CDFA

# Predict

In [ ]:
if __name__ == '__main__':
    opt = TestOptions().parse()   # get training options
    opt = make_val_opt(opt)
    opt.phase = 'test'
    opt.dataroot = 'path-to-LEVIR-CD-test' # data root 
    opt.dataset_mode = 'changedetection'
    opt.n_class = 2
    opt.SA_mode = 'PAM' # BAM | PAM 
    opt.arch = 'mynet3'
    opt.model = 'CDFA' # model type
    opt.name = 'LEVIR-CDFAp0' # project name
    opt.results_dir = './results/' # save predicted images 
    opt.epoch = 'best-epoch-in-val' # which epoch to test
    opt.num_test = np.inf
    val(opt)